# QAOA and QAA to solve a QUBO problem

In [ ]:
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
import pulser
import pulser_simulation
from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

## Introduction

In this tutorial, we illustrate how to solve a Quadratic Unconstrained Binary Optimization (QUBO) instance using an ensemble of Rydberg atoms in analog mode.

QUBO has been extensively studied [Glover, et al., 2018](https://arxiv.org/pdf/1811.11538.pdf) and is used to model and solve numerous categories of optimization problems including important instances of network flows, scheduling, max-cut, max-clique, vertex cover and other graph and management science problems, integrating them into a unified modeling framework.

Mathematically, a QUBO instance consists of a symmetric matrix $Q$ of size $(N \times N)$, and the optimization problem associated with it is to find the bitstring $z=(z_1, \dots, z_N) \in \{0, 1 \}^N$ that minimizes the quantity
$$f(z) = z^{T}Qz$$ 

Suppose we are given the following QUBO matrix $Q$:

In [ ]:
Q = np.array(
    [
        [-10.0, 19.7365809, 19.7365809, 5.42015853, 5.42015853],
        [19.7365809, -10.0, 20.67626392, 0.17675796, 0.85604541],
        [19.7365809, 20.67626392, -10.0, 0.85604541, 0.17675796],
        [5.42015853, 0.17675796, 0.85604541, -10.0, 0.32306662],
        [5.42015853, 0.85604541, 0.17675796, 0.32306662, -10.0],
    ]
)

In this tutorial, we will demonstrate how this QUBO instance can be mapped and solved using neutral atoms. For reasons that will become apparent further along, this QUBO instance is particularly amenable to embedding on a neutral-atom device since:

1. All the off-diagonal terms are positive.
2. The diagonal terms are all equal.

Furthermore, because the QUBO is small, we can classically check all solutions and mark the optimal ones. This will help us later in the tutorial to visualize the quality of our quantum approach.

In [ ]:
bitstrings = [np.binary_repr(i, len(Q)) for i in range(2 ** len(Q))]
costs = []
# this takes exponential time with the dimension of the QUBO
for b in bitstrings:
    z = np.array(list(b), dtype=int)
    cost = z.T @ Q @ z
    costs.append(cost)
zipped = zip(bitstrings, costs)
sort_zipped = sorted(zipped, key=lambda x: x[1])
print(sort_zipped[:3])

This QUBO admits `01011` and `00111` as optimal solutions.

## Solving the QUBO with `Pulser` 

Let's remind that Pulser enables you to program the Ising Hamiltonian $H_Q$:

$$\frac{H_Q}{\hbar}(t) = \sum_{k=1}^N \left (\frac{\Omega(t)}{2} e^{-i\phi(t)} |g\rangle\langle r|_k + \frac{\Omega(t)}{2} e^{i\phi(t)} |r\rangle\langle g|_k - \delta(t) |r\rangle\langle r|_k(t) + \sum_{j<k}\frac{C_6}{\hbar R_{kj}^6} \hat{n}_k \hat{n}_j \right)
$$

The key idea is to encode the off-diagonal terms of $Q$ by using the Rydberg interaction between atoms: 
$$\left[\frac{C_6}{\hbar R_{kj}^6}\right]_{1\leq k, h \leq N}\leftrightarrow Q_{off-diagonal}$$

Since the Rydberg interaction depends on the pairwise distance between atoms $k$ and $j$, $R_{kj}$, this will define the `Register` of atoms to use in the computation.

The second idea is to find the optimal solution of the QUBO by preparing the ground-state of $H_Q$ and outputing the optimal bitstrings. There are multiple approaches to prepare the ground-state of an Hamiltonian. We will here present two different approaches, the Quantum Approximation Optimization Algorithm (QAOA) and the Quantum Adiabatic Algorithm (QAA). We will see that the QAA approach is more suited for cold-atom quantum computers. 

<div class="alert alert-info">

**Note**:

1. The Rydberg interaction is strictly positive, which is why it matters that our off-diagonal terms are too.
2. The diagonal terms of the $Q$ are understood as a detuning to apply on each atom. Since the diagonal terms are all the same, we can use the same detuning  $\delta$ for all the atoms. If that was not the case, some kind of local addressability capabilities would be necessary (see [Weighted Analog Quantum Computing](../../docs/source/extended_usage_intro.md#weighted-analog-quantum-computing)).

</div>



 

### 1. Pick a Device

To implement the Hamiltonian $H_Q$, we need a `Device` containing a `Rydberg.Global` channel. `pulser.AnalogDevice` and `pulser.DigitalAnalogDevice` are examples of `Devices` that contain a `Rydberg.Global` channel. Since the durations of the pulses will be quite long, let's use `pulser.DigitalAnalogDevice` since it allows the creation of longer sequences.

In [ ]:
device = pulser.DigitalAnalogDevice
device.print_specs()

### 2. Create the Register

Let's attempt a simple minimization procedure to find the optimal positions of the atoms in the Register that replicate best the off-diagonal terms of $Q$:

In [ ]:
def evaluate_mapping(
    new_coords: np.ndarray, Q: np.ndarray, device: pulser.devices.Device
):
    """Cost function to minimize. Ideally, the pairwise distances are conserved."""
    new_coords = np.reshape(new_coords, (len(Q), 2))
    # computing the matrix of the distances between all coordinate pairs
    new_Q = squareform(device.interaction_coeff / pdist(new_coords) ** 6)
    return np.linalg.norm(new_Q - Q)

In [ ]:
costs = []
np.random.seed(0)
x0 = np.random.random(len(Q) * 2)
res = minimize(
    evaluate_mapping,
    x0,
    args=(Q, device),
    method="Nelder-Mead",
    tol=1e-6,
    options={"maxiter": 200000, "maxfev": None},
)
coords = np.reshape(res.x, (len(Q), 2))

We can then build the `Register` from the obtained coordinates:

In [ ]:
qubits = {f"q{i}": coord for (i, coord) in enumerate(coords)}
reg = pulser.Register(qubits)
reg.draw(
    blockade_radius=device.rydberg_blockade_radius(1.0),
    draw_graph=False,
    draw_half_radius=True,
)

Let's check that this register matches with the device's constraints by initializing the `Sequence`: 

In [ ]:
sequence = pulser.Sequence(reg, device)

<div class="alert alert-warning">

**Note**:

In this case, this simple procedure was enough to give a good and valid embedding but it will not always be so. For QUBO instances that are not as easy to embbed as this one, more complex embedding strategies are required.

</div>



### 3. Pick the channels

The channel needed to implement the Ising Hamiltonian $H_Q$ is the `Rydberg.Global` channel. Let's declare this channel in the `Sequence`:

In [ ]:
sequence.declare_channel("rydberg_global", "rydberg_global")

### 4. Add the Pulses

The value of the amplitude, detuning and phase along time in $H_Q$ will be determined by the approach used to prepare the ground-state of the Hamiltonian:  

#### QAOA for depth $p=2$

##### QAOA parametrized Sequence

This algorithm (see [Farhi, et al., 2014](https://arxiv.org/pdf/1411.4028.pdf)) has gained a lot of traction lately as a gate-based quantum algorithm. It has shown promising results in a number of applications and yields decent results for low-depth circuits.

All atoms are initially in the groundstate $|00\dots0\rangle$ of the `ground-rydberg` basis.  We then apply $p$ layers of alternating non-commutative Hamiltonians. The first one, called the mixing Hamiltonian $H_M$, is realized by taking $\Omega = 1$ rad/µs, and $\delta = 0$ rad/µs in the Hamiltonian equation. The second Hamiltonian $H_Q$ is realized with $\Omega =0$ rad/µs and $\delta = 1.$ rad/µs. $H_M$ and $H_Q$ are applied turn in turn with parameters $\tau$ and $t$ respectively. A classical optimizer is then used to estimate the optimal parameters. 

<div class="alert alert-info">

Instead of creating a new `Sequence` everytime the quantum loop is called, we are going to create a parametrized `Sequence` and give that to the quantum loop.

</div>

In [ ]:
qaoa_seq = copy(sequence)  # a specific sequence for qaoa
LAYERS = 2

# Declare variables to parametrize the Sequence
t_list = qaoa_seq.declare_variable("t_list", size=LAYERS)
s_list = qaoa_seq.declare_variable("s_list", size=LAYERS)

for t, s in zip(t_list, s_list):
    pulse_1 = pulser.Pulse.ConstantPulse(1000 * t, 1.0, 0.0, 0)
    pulse_2 = pulser.Pulse.ConstantPulse(1000 * s, 0.0, 1.0, 0)

    qaoa_seq.add(pulse_1, "rydberg_global")
    qaoa_seq.add(pulse_2, "rydberg_global")

qaoa_seq.measure("ground-rydberg")

Once we have the parameters that we want to apply, we use the `.build()` method to assign these values into a `assigned_seq` sequence. It is this sequence which is submitted to the backend in the end.

In [ ]:
def quantum_loop(parameters, draw: bool = False):
    params = np.array(parameters)
    t_params, s_params = np.reshape(params.astype(int), (2, LAYERS))
    assigned_seq = qaoa_seq.build(t_list=t_params, s_list=s_params)
    if draw:
        assigned_seq.draw()
    simul = pulser_simulation.QutipBackend(
        assigned_seq, pulser.EmulatorConfig(sampling_rate=0.01)
    )
    results = simul.run()
    count_dict = results.sample_final_state()  # sample from the state vector
    return count_dict

##### Evaluating the Sequence with random parameters

We can evaluate the parametrized with random parameters:

In [ ]:
np.random.seed(123)  # ensures reproducibility of the tutorial
guess = {
    "t": np.random.uniform(8, 10, LAYERS),
    "s": np.random.uniform(1, 3, LAYERS),
}
# Get the count for these parameters and draw the Sequence
example_count = quantum_loop(np.r_[guess["t"], guess["s"]], draw=True)

Let's plot the distribution of the samples, to see the most frequent bitstrings sampled.

In [ ]:
def plot_distribution(C):
    C = dict(sorted(C.items(), key=lambda item: item[1], reverse=True))
    indexes = ["01011", "00111"]  # QUBO solutions
    color_dict = {key: "r" if key in indexes else "g" for key in C}
    plt.figure(figsize=(12, 6))
    plt.xlabel("bitstrings")
    plt.ylabel("counts")
    plt.bar(C.keys(), C.values(), width=0.5, color=color_dict.values())
    plt.xticks(rotation="vertical")
    plt.show()

In [ ]:
plot_distribution(example_count)

The bitstrings `01011` and `00111` (in red) correspond to the two optimal solutions (calculated at the beginning of the notebook). Since the parameters of the sequence were chosen randomly, the optimal solutions were not found. The goal of QAOA is to choregraph interferences between the basis states, in order to maximize the frequency of the optimal solution states. 

##### Optimize the parameters

We estimate the cost of a sampled state vector by making an average over the samples. This is done by taking the corresponding bitstring ${\bf z}=(z_1, \ldots, z_N)$ and calculating

$$
C({\bf z}) = {\bf z}^\top \cdot Q \cdot {\bf z}
$$

Determining the cost of a given bitstring takes polynomial time. The average estimate is then used in the classical loop to optimize the variational parameters $\tau$ and $t$.

In [ ]:
def get_cost_colouring(bitstring, Q):
    z = np.array(list(bitstring), dtype=int)
    cost = z.T @ Q @ z
    return cost


def get_cost(counter, Q):
    cost = sum(counter[key] * get_cost_colouring(key, Q) for key in counter)
    return cost / sum(counter.values())  # Divide by total samples

To perform a minimization loop and find the best variational parameters, we use the classical optimizer `minimize`.

It takes as input:
- A function that takes as input the QUBO (first argument of `*args`) and the trial value to score (`params`), and returns the cost associated with this QUBO and these values.
- the QUBO $Q$.
- an initial point `x0` for the simplex in Nelder-Mead minimization

As the optimizer might get trapped in local minima, we repeat the optimization 20 times and select the parameters that yield the best approximation ratio.

In [ ]:
def func(param, *args):
    Q = args[0]
    C = quantum_loop(param)
    cost = get_cost(C, Q)
    return cost

In [ ]:
scores = []
params = []
for repetition in range(20):
    guess = {
        "t": np.random.uniform(1, 10, LAYERS),
        "s": np.random.uniform(1, 10, LAYERS),
    }

    try:
        res = minimize(
            func,
            args=Q,
            x0=np.r_[guess["t"], guess["s"]],
            method="Nelder-Mead",
            tol=1e-5,
            options={"maxiter": 10},
        )
        scores.append(res.fun)
        params.append(res.x)
    except Exception as e:
        pass

Let's plot the samples that we woud obtain using the optimal variational parameters.

In [ ]:
optimal_count_dict = quantum_loop(params[np.argmin(scores)])
plot_distribution(optimal_count_dict)

QAOA is capable of finding good variational parameters $\tau$ and $t$. Now, sampling from this final state $|\psi(t_{f})\rangle$ will return both optimal strings with high probability.

However, using QAOA to solve the problem is not the best idea; it's difficult to yield a >90% quality solution without going to high depths of the QAOA, implying that the growing closed-loop optimization can rapidly become expensive, with no guarantee of convergence. We therefore propose another approach called the Quantum Adiabatic Algorithm (QAA). This fast, reliant and exclusively analog method shows optimal convergence to the solution.

#### Quantum Adiabatic Algorithm

The idea behind the adiabatic algorithm (see [Albash, Lidar, 2018](https://arxiv.org/pdf/1611.04471.pdf)) is to slowly evolve the system from an easy-to-prepare groundstate to the groundstate of $H_Q$. If done slowly enough, the system of atoms stays in the instantaneous ground-state.

In our case, we continuously vary the parameters $\Omega(t), \delta(t)$ in time, starting with $\Omega(0)=0, \delta(0)<0$ and ending with $\Omega(0)=0, \delta>0$. The ground-state of $H(0)$ corresponds to the initial state $|ggggg\rangle$ and the ground-state of $H(t_f)$ corresponds to the ground-state of $H_Q$.

To ensure that we are not exciting the system to states that are too excited, we keep $\Omega \in [0, \Omega_{\text{max}}]$, and choose $\Omega_{\text{max}}$ as the median of the values of Q to ensures that the adiabatic path is efficient.

In [ ]:
# We choose a median value between the min and the max
Omega = np.median(Q[Q > 0].flatten())
delta_0 = -5  # just has to be negative
delta_f = -delta_0  # just has to be positive
T = 4000  # time in ns, we choose a time long enough to ensure the propagation of information in the system

In [ ]:
qaa_seq = copy(sequence)
adiabatic_pulse = pulser.Pulse(
    pulser.InterpolatedWaveform(T, [1e-9, Omega, 1e-9]),
    pulser.InterpolatedWaveform(T, [delta_0, 0, delta_f]),
    0,
)
qaa_seq.add(adiabatic_pulse, "rydberg_global")
qaa_seq.draw()

In [ ]:
simul = pulser_simulation.QutipBackend(qaa_seq)
results = simul.run()
final = results.get_final_state()
count_dict = results.sample_final_state()

In [ ]:
plot_distribution(count_dict)

See how fast and performant this method is! In only a few micro-seconds, we find an excellent solution.

### How does the time evolution affect the quality of the results?

In [ ]:
cost = []
for T in 1000 * np.linspace(1, 10, 10):
    seq = pulser.Sequence(reg, pulser.DigitalAnalogDevice)
    seq.declare_channel("ising", "rydberg_global")
    adiabatic_pulse = pulser.Pulse(
        pulser.InterpolatedWaveform(T, [1e-9, Omega, 1e-9]),
        pulser.InterpolatedWaveform(T, [delta_0, 0, delta_f]),
        0,
    )
    seq.add(adiabatic_pulse, "ising")
    simul = pulser_simulation.QutipBackend(seq)
    results = simul.run()
    final = results.get_final_state()
    count_dict = results.sample_final_state()
    cost.append(get_cost(count_dict, Q) / 3)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1, 11), np.array(cost), "--o")
plt.xlabel("total time evolution (µs)", fontsize=14)
plt.ylabel("cost", fontsize=14)
plt.show()